<a href="https://colab.research.google.com/github/rtrochepy/astronomer/blob/main/best_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, balanced_accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from joblib import dump, load
from scipy.stats import skew, kurtosis
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTEENN
import os
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import numpy as np

In [4]:
# Ignorar advertencias de pandas para una ejecución más limpia
warnings.filterwarnings('ignore')

# Configuración de pandas para mostrar todas las columnas al imprimir
pd.set_option('display.max_columns', None)

In [5]:
# Lee el archivo CSV.  Error handling mejorado.
try:
    df = pd.read_csv("test_data.csv")
except FileNotFoundError:
    print("Error: El archivo 'data_labels.csv' no se encuentra.")
except pd.errors.EmptyDataError:
    print("Error: El archivo 'data_labels.csv' está vacío.")
except pd.errors.ParserError:
    print("Error: Error al analizar el archivo 'data_labels.csv'.")

In [6]:
# Ver los nulos que hay en cada columna
nulos_por_columna = df.isnull().sum()

# Filtrar solo las columnas que tienen valores nulos
nulos_por_columna = nulos_por_columna[nulos_por_columna > 0]
nulos_por_columna

,0
Payment_6804,399
Base_80863,15
Expenditure_JIG,11632
Infraction_SNZ,15
Base_02683,15
...,...
Infraction_QGR,968
Infraction_ZTLC,49935
Infraction_LSX,968
Infraction_IBJ,413


In [7]:
# Calcula el umbral para filtrar las columnas
umbral = 0.5 * df.shape[0]

# Elimina las columnas con más del 50% de valores faltantes
df = df.dropna(thresh=umbral, axis=1)
df

,ID,Expenditure_AHF,Payment_6804,Infraction_CGP,Base_7744,Base_80863,Risk_1930,Expenditure_JIG,Infraction_SNZ,Base_02683,Infraction_ZWWJ,Infraction_QJJF,Base_76065,Infraction_EJZ,Base_6872,Risk_0322,Infraction_FMXQ,Infraction_GGO,Infraction_TLPJ,Base_1165,Base_39598,Base_6187,Infraction_ZTNC,Base_85131,Risk_9995,Infraction_AYWV,Payment_22507,Base_9516,Expenditure_YTR,Base_36384,Expenditure_FIP,Infraction_PAS,Risk_0003,Expenditure_HMO,Base_24406,Expenditure_LMSR,Infraction_BSU,Base_14808,Risk_8065,Infraction_ZYW,Base_1039,Infraction_HSSU,Infraction_EHZP,Infraction_TBP,Base_0580,Expenditure_RGD,Infraction_PBC,Infraction_YFSG,Infraction_DQLY,Infraction_AQO,Base_0229,Base_69608,Base_91828,Base_6852,Infraction_CLH,Expenditure_IDZ,Risk_1475,Expenditure_BWX,Base_8511,Infraction_JYZB,Base_22178,Infraction_ZTYG,Infraction_ZVW,Infraction_EYU,Expenditure_UWVG,Base_3041,Payment_3207,Infraction_QKZN,Infraction_CZE,Base_65352,Risk_7095,Infraction_JBR,Base_66195,Base_36516,Infraction_RXQH,Infraction_HFU,Risk_6346,Expenditure_HRQ,Infraction_VTR,Risk_2102,Risk_4804,Base_7331,Infraction_XWX,Expenditure_XDD,Risk_4553,Base_67585,Risk_8742,Infraction_VHU,Risk_4247,Risk_2380,Infraction_GSS,Risk_0454,Base_8730,Expenditure_HKXV,Infraction_MHM,Risk_4160,Risk_3506,Base_23737,Expenditure_GCAO,Risk_9367,Base_7910,Expenditure_GMC,Risk_9423,Risk_6977,Base_9103,Infraction_KSBR,Risk_6178,Risk_6197,Infraction_NRBQ,Infraction_WVC,Infraction_QVSL,Infraction_QXUM,Risk_8532,Risk_9247,Infraction_IMIM,Expenditure_UIWS,Expenditure_ONEG,Expenditure_MTRQ,Expenditure_LAHK,Expenditure_HPM,Infraction_LTIS,Infraction_HFSI,Infraction_ETH,Infraction_SDWM,Base_5441,Base_2810,Risk_8902,Base_67254,Infraction_PTY,Infraction_BGGU,Base_4569,Expenditure_BEH,Infraction_LMHK,Infraction_NMCB,Infraction_TPAF,Infraction_ZRH,Infraction_TEN,Infraction_XEPQ,Infraction_ZMKI,Infraction_WIS,Infraction_RKTA,Infraction_IIZ,Infraction_WVAW,Infraction_KEJT,Infraction_TFOY,Infraction_WMAQ,Infraction_SIA,Infraction_CZXL,Infraction_QEY,Base_52892,Infraction_HUK,Infraction_VHHP,Infraction_LIES,Risk_5270,Infraction_QWWW,Infraction_YQXM,Infraction_QGR,Infraction_LSX,Infraction_IBJ,Infraction_DNOU
0,3333333702723732807074333342320276353770732523...,2017-03-15,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,NaN,0.000630,0.080986,0.708906,0.170600,0.006204,0.358587,0.525351,0.255736,0.063902,0.059416,0.006466,1.335856,0.008207,0.001423,0.207334,0.736463,0.096219,0.023381,0.002768,0.008322,1.001519,0.008298,0.161345,0.148266,0.922998,0.354596,0.118075,0.001882,0.158612,0.018385,0.063646,0.199617,0.308233,0.016361,0.401619,0.091071,CR,O,0.007126,0.007665,0.652984,0.008520,0.004730,very_high,0.272008,0.008363,0.515222,0.002644,0.009013,0.004808,0.008342,0.119403,0.004802,0.108271,0.050882,0.007554,0.080422,0.069067,0.004327,0.007562,NaN,0.007729,0.000272,0.001576,0.004239,0.001434,0.002271,0.004061,0.007121,0.002456,0.002310,0.003532,0.008033,1.009825,0.084683,0.003820,0.007043,0.000438,0.006452,0.000830,0.005055,0.0,0.005720,0.007084,0.000198,0.008907,1,0.002537,0.005177,0.006626,0.009705,0.007782,0.002450,1.001101,0.002665,0.007479,0.006893,1.503673,1.006133,0.003569,0.008871,0.003950,0.003647,0.004950,0.894090,0.135561,0.911191,0.974539,0.001243,0.766688,1.008691,1.004587,0.670041,0.009968,0.004572,1.008949,moderate_low,0.004326,1.007336,0.210060,0.676922,0.007871,1.0,0.238250,0.0,moderate_high,0.232120,0.236266,0.0,0.702280,0.434345,0.003057,0.686516,0.008740,1.0,1.003319,1.007819,1.000080,0.006805,0.002052,0.005972,0.004345,0.001535,0.002427,0.003706,0.003818,0.000569,0.000610,0.002674
1,3333333702723732807074333342320276353770732523...,2017-04-13,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,NaN,0.002526,0.069419,0.712795,0.113239,0.006206,0.353630,0.521311,0.223329,0.065261,0.057744,0.001614,1.339794,0.008373,0.001984,0.202778,0.720886,0.099804,0.030599,0.002749,0.002482,1.009033,0.005136,0.140951,0.143530,0.919414,0.326757,0.118737,0.001610,0.148459,

In [8]:
# 1. Definir columnas no numéricas y columnas numéricas
no_numerical = ['ID', 'Expenditure_AHF', 'Infraction_YFSG', 'Infraction_DQLY', 'Infraction_CLH',
                'Base_67254', 'Infraction_TEN', 'Base_8730', 'Base_23737', 'Infraction_NMCB',
                'Infraction_ZRH', 'Infraction_WIS', 'Infraction_WMAQ', 'label']

# Seleccionar solo las columnas numéricas
numerical_columns = df.select_dtypes(include=['number']).columns
numerical_columns = [col for col in numerical_columns if col not in no_numerical]

# Crear una copia del DataFrame para evitar advertencias
df_clean = df.copy()

# 2. Calcular el porcentaje de valores nulos en columnas numéricas
porcentaje_null = df_clean[numerical_columns].isnull().mean() * 100

# 3. Clasificar columnas según el porcentaje de valores nulos
low_null_columns = porcentaje_null[porcentaje_null < 10].index
mid_null_columns = porcentaje_null[(porcentaje_null >= 10) & (porcentaje_null < 30)].index
high_null_columns = porcentaje_null[porcentaje_null >= 30].index

# Mostrar un informe
print("Resumen del Tratamiento de Valores Nulos:")
print(f"- Columnas con menos del 10% de nulos: {len(low_null_columns)} columnas.")
print(f"- Columnas con entre 10% y 30% de nulos: {len(mid_null_columns)} columnas.")
print(f"- Columnas con más del 30% de nulos: {len(high_null_columns)} columnas.\n")

# 4. Tratamiento de Valores Nulos
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Menos del 10% de nulos - rellenar con la media
if len(low_null_columns) > 0:
    df_clean[low_null_columns] = df_clean[low_null_columns].fillna(df_clean[low_null_columns].mean())
    print("✔ Columnas con menos del 10% de nulos tratadas con la media.")

# Entre 10% y 30% de nulos - rellenar con la mediana
if len(mid_null_columns) > 0:
    df_clean[mid_null_columns] = df_clean[mid_null_columns].fillna(df_clean[mid_null_columns].median())
    print("✔ Columnas con entre 10% y 30% de nulos tratadas con la mediana.")

# Más del 30% de nulos - usar IterativeImputer
if len(high_null_columns) > 0:
    imputer = IterativeImputer(random_state=42)
    df_clean[high_null_columns] = imputer.fit_transform(df_clean[high_null_columns])
    print("✔ Columnas con más del 30% de nulos tratadas con Iterative Imputer.")

# Verificación final de valores nulos
print("\nValores nulos después del tratamiento:")
print(df_clean[numerical_columns].isnull().sum().sort_values(ascending=False))

Resumen del Tratamiento de Valores Nulos:
- Columnas con menos del 10% de nulos: 138 columnas.
- Columnas con entre 10% y 30% de nulos: 8 columnas.
- Columnas con más del 30% de nulos: 1 columnas.

✔ Columnas con menos del 10% de nulos tratadas con la media.
✔ Columnas con entre 10% y 30% de nulos tratadas con la mediana.
✔ Columnas con más del 30% de nulos tratadas con Iterative Imputer.

Valores nulos después del tratamiento:
Payment_6804        0
Expenditure_MTRQ    0
Expenditure_GMC     0
Risk_9423           0
Risk_6977           0
                   ..
Base_6852           0
Expenditure_IDZ     0
Risk_1475           0
Expenditure_BWX     0
Infraction_DNOU     0
Length: 147, dtype: int64


In [9]:
# 1. Variables categóricas
categoricas = ['Infraction_YFSG', 'Infraction_DQLY', 'Infraction_CLH', 'Base_67254',
               'Infraction_TEN', 'Base_8730', 'Base_23737', 'Infraction_NMCB',
               'Infraction_ZRH', 'Infraction_WIS', 'Infraction_WMAQ']

# 2. Mostrar categorías únicas para cada variable categórica
print("Categorías únicas antes de rellenar valores faltantes:")
for col in categoricas:
    if col in df.columns:
        print(f"{col}: {df[col].unique()}")
    else:
        print(f"Advertencia: La columna '{col}' no existe en el DataFrame.")

# 3. Rellenar valores faltantes con 'Missing'
for col in categoricas:
    if col in df.columns:
        df[col].fillna('Missing', inplace=True)
        print(f"✔ Valores faltantes en '{col}' reemplazados con 'Missing'.")
    else:
        print(f"❌ La columna '{col}' no fue encontrada.")

# 4. Comprobar que los valores faltantes se han rellenado
print("\nCategorías únicas después de rellenar valores faltantes:")
for col in categoricas:
    if col in df.columns:
        print(f"{col}: {df[col].unique()}")

# 5. Convertir columnas categóricas a tipo 'category'
df[categoricas] = df[categoricas].astype('category')
print("\n✔ Columnas categóricas convertidas a tipo 'category' en pandas.")

# 6. Función para aplicar binning con 'Missing'
def binarize_levels_with_missing(value):
    """
    Función para agrupar niveles en categorías más amplias.
    """
    if value == 'Missing':
        return 'Missing'
    elif value in ['very_low', 'low', 'moderate_low', 'extremely_low']:
        return 'low'
    elif value in ['moderate', 'moderate_high']:
        return 'moderate'
    else:
        return 'high'

# 7. Aplicar binning a cada columna categórica
for col in categoricas:
    if col in df.columns:
        df[col + "_binned"] = df[col].apply(binarize_levels_with_missing)
        print(f"✔ Binning aplicado a la columna '{col}', nueva columna: '{col}_binned'.")

# 8. Verificar el resultado final
print("\nEjemplo de binning en las primeras filas:")
for col in categoricas:
    if col in df.columns:
        print(f"\n{col} (original) -> {col}_binned (agrupada):")
        print(df[[col, col + "_binned"]].head())

Categorías únicas antes de rellenar valores faltantes:
Infraction_YFSG: ['CR' 'CO' 'CL' 'XL' 'XM' 'XZ']
Infraction_DQLY: ['O' 'R' nan 'U' '-1']
Infraction_CLH: ['very_high' nan 'moderate_low' 'moderate' 'high' 'moderate_high' 'low'
 'very_low']
Base_67254: ['moderate_low' 'low' 'high' 'moderate' 'moderate_high' 'very_high'
 'extremely_high' nan]
Infraction_TEN: ['moderate_high' 'extremely_low' nan 'moderate_low' 'very_high' 'moderate'
 'high' 'low']
Base_8730: [ 0.  2.  1. nan]
Base_23737: [1 0]
Infraction_NMCB: [ 1.  0. nan]
Infraction_ZRH: [ 0. nan  1.]
Infraction_WIS: [ 0. nan  1.]
Infraction_WMAQ: [ 1. nan  0. -1.]
✔ Valores faltantes en 'Infraction_YFSG' reemplazados con 'Missing'.
✔ Valores faltantes en 'Infraction_DQLY' reemplazados con 'Missing'.
✔ Valores faltantes en 'Infraction_CLH' reemplazados con 'Missing'.
✔ Valores faltantes en 'Base_67254' reemplazados con 'Missing'.
✔ Valores faltantes en 'Infraction_TEN' reemplazados con 'Missing'.
✔ Valores faltantes en 'Base_8730' 

In [10]:
from sklearn.preprocessing import LabelEncoder

# Función para aplicar binning a columnas con orden implícito
def apply_binning(df, columns, binarize_func):
    """
    Aplica una función de binning a columnas seleccionadas.
    """
    for col in columns:
        if col in df.columns:
            df[col] = df[col].apply(binarize_func)
            print(f"✔ Binning aplicado en '{col}'.")
        else:
            print(f"❌ La columna '{col}' no existe.")
    return df

# Función para aplicar One-Hot Encoding
def apply_one_hot_encoding(df, columns):
    """
    Aplica One-Hot Encoding a columnas seleccionadas.
    """
    if any(col in df.columns for col in columns):
        df = pd.get_dummies(df, columns=columns, drop_first=False)
        print(f"✔ One-Hot Encoding aplicado a columnas: {columns}.")
    else:
        print(f"❌ Columnas no encontradas: {columns}.")
    return df

# Función para aplicar Label Encoding a columnas binarias
def apply_label_encoding(df, columns):
    """
    Aplica Label Encoding a columnas binarias después de convertirlas a string.
    """
    le = LabelEncoder()
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype(str)
            df[col] = le.fit_transform(df[col])
            print(f"✔ Label Encoding aplicado en '{col}'.")
        else:
            print(f"❌ La columna '{col}' no existe.")
    return df

# Función para verificar valores nulos
def check_missing_values(df):
    """
    Verifica y muestra columnas con valores nulos.
    """
    nulos = df.isnull().sum()
    nulos = nulos[nulos > 0]
    if nulos.empty:
        print("✔ No hay valores nulos en el DataFrame.")
    else:
        print("❌ Columnas con valores nulos:")
        print(nulos)

# Aplicar binning a columnas con orden implícito
columns_with_order = ['Infraction_CLH', 'Base_67254', 'Infraction_TEN']
df = apply_binning(df, columns_with_order, binarize_levels_with_missing)

# Aplicar One-Hot Encoding a columnas seleccionadas
one_hot_columns = {
    'ordered_columns': ['Infraction_CLH', 'Base_67254', 'Infraction_TEN'],
    'unordered_columns': ['Infraction_YFSG', 'Infraction_DQLY'],
    'numeric_low_cardinality': ['Base_8730', 'Infraction_WMAQ']
}

for key, cols in one_hot_columns.items():
    df = apply_one_hot_encoding(df, cols)

# Aplicar Label Encoding a columnas binarias
binary_columns = ['Base_23737', 'Infraction_NMCB', 'Infraction_ZRH', 'Infraction_WIS']
df = apply_label_encoding(df, binary_columns)

# Convertir la columna 'Expenditure_AHF' a datetime
if 'Expenditure_AHF' in df.columns:
    df['Expenditure_AHF'] = pd.to_datetime(df['Expenditure_AHF'], errors='coerce')
    print("✔ Columna 'Expenditure_AHF' convertida a datetime.")
else:
    print("❌ La columna 'Expenditure_AHF' no existe.")

# Establecer la columna 'ID' como índice
if 'ID' in df.columns:
    df.set_index('ID', inplace=True)
    print("✔ Columna 'ID' establecida como índice.")
else:
    print("❌ La columna 'ID' no existe.")

# Verificar nulos después de las transformaciones
print("\nVerificación final de valores nulos:")
check_missing_values(df)

# Mostrar las primeras filas para verificar
print("\nPrimeras filas del DataFrame transformado:")
print(df.head())

✔ Binning aplicado en 'Infraction_CLH'.
✔ Binning aplicado en 'Base_67254'.
✔ Binning aplicado en 'Infraction_TEN'.
✔ One-Hot Encoding aplicado a columnas: ['Infraction_CLH', 'Base_67254', 'Infraction_TEN'].
✔ One-Hot Encoding aplicado a columnas: ['Infraction_YFSG', 'Infraction_DQLY'].
✔ One-Hot Encoding aplicado a columnas: ['Base_8730', 'Infraction_WMAQ'].
✔ Label Encoding aplicado en 'Base_23737'.
✔ Label Encoding aplicado en 'Infraction_NMCB'.
✔ Label Encoding aplicado en 'Infraction_ZRH'.
✔ Label Encoding aplicado en 'Infraction_WIS'.
✔ Columna 'Expenditure_AHF' convertida a datetime.
✔ Columna 'ID' establecida como índice.

Verificación final de valores nulos:
❌ Columnas con valores nulos:
Payment_6804         399
Base_80863            15
Expenditure_JIG    11632
Infraction_SNZ        15
Base_02683            15
                   ...  
Infraction_YQXM      399
Infraction_QGR       968
Infraction_LSX       968
Infraction_IBJ       413
Infraction_DNOU      968
Length: 78, dtype: 

In [11]:
import pandas as pd
import numpy as np
from scipy.stats import skew

# 1. Función para calcular IQR y límites
def calculate_iqr_limits(df, columns):
    """
    Calcula Q1, Q3 e IQR para columnas numéricas.
    Retorna límites inferior y superior para detección de outliers.
    """
    Q1 = df[columns].quantile(0.25)
    Q3 = df[columns].quantile(0.75)
    IQR = Q3 - Q1
    lower_limits = Q1 - 1.5 * IQR
    upper_limits = Q3 + 1.5 * IQR
    return lower_limits, upper_limits

# 2. Función para identificar y clasificar outliers
def classify_outliers(df, columns, lower_limits, upper_limits):
    """
    Clasifica columnas según porcentaje de outliers: <5%, 5%-15%, >15%.
    """
    outliers = ((df[columns] < lower_limits) | (df[columns] > upper_limits)).sum()
    total_rows = len(df)
    outlier_percentage = (outliers / total_rows) * 100

    # Clasificar columnas
    minor = outlier_percentage[outlier_percentage < 5].index
    moderate = outlier_percentage[(outlier_percentage >= 5) & (outlier_percentage <= 15)].index
    high = outlier_percentage[outlier_percentage > 15].index

    print("\nResumen de Outliers:")
    print(f"✔ Columnas con < 5% de outliers: {list(minor)}")
    print(f"✔ Columnas con 5%-15% outliers: {list(moderate)}")
    print(f"✔ Columnas con > 15% outliers: {list(high)}")
    return minor, moderate, high

# 3. Función para manejar outliers
def handle_outliers(df, minor, moderate, high, lower_limits, upper_limits):
    """
    Elimina, reemplaza o transforma outliers en función de su clasificación.
    """
    # Eliminar outliers en columnas con menos del 5%
    for col in minor:
        df = df[(df[col] >= lower_limits[col]) & (df[col] <= upper_limits[col])]
        print(f"✔ Outliers eliminados en '{col}'.")

    # Reemplazar outliers con límites en columnas moderadas
    for col in moderate:
        df[col] = np.clip(df[col], lower_limits[col], upper_limits[col])
        print(f"✔ Outliers reemplazados con límites en '{col}'.")

    # Transformar columnas con alto número de outliers
    for col in high:
        original_skew = skew(df[col].dropna())
        log_skew = skew(np.log1p(df[col].dropna()))
        sqrt_skew = skew(np.sqrt(df[col].clip(lower=0)))

        if abs(log_skew) < abs(sqrt_skew):
            df[col] = np.log1p(df[col].clip(lower=0))
            print(f"✔ Transformación logarítmica aplicada en '{col}'.")
        else:
            df[col] = np.sqrt(df[col].clip(lower=0))
            print(f"✔ Transformación raíz cuadrada aplicada en '{col}'.")

    return df

# 4. Flujo Principal
# Seleccionar columnas numéricas
numeric_columns = df.select_dtypes(include=np.number).columns

# Calcular límites de IQR
lower_limits, upper_limits = calculate_iqr_limits(df, numeric_columns)

# Clasificar columnas por porcentaje de outliers
minor, moderate, high = classify_outliers(df, numeric_columns, lower_limits, upper_limits)

# Manejar outliers
df_cleaned = handle_outliers(df, minor, moderate, high, lower_limits, upper_limits)

# Verificación final
print("\nVerificación de valores nulos después del tratamiento:")
print(df_cleaned.isnull().sum().sort_values(ascending=False))

# Mostrar primeras filas
print("\nPrimeras filas del DataFrame limpio:")
print(df_cleaned.head())


Resumen de Outliers:
✔ Columnas con < 5% de outliers: ['Payment_6804', 'Base_80863', 'Infraction_EJZ', 'Risk_0322', 'Infraction_GGO', 'Infraction_TLPJ', 'Base_1165', 'Base_6187', 'Infraction_ZTNC', 'Base_85131', 'Infraction_AYWV', 'Base_9516', 'Infraction_PAS', 'Risk_0003', 'Expenditure_HMO', 'Expenditure_LMSR', 'Infraction_BSU', 'Risk_8065', 'Infraction_ZYW', 'Infraction_HSSU', 'Infraction_EHZP', 'Infraction_TBP', 'Infraction_PBC', 'Infraction_AQO', 'Base_0229', 'Base_69608', 'Risk_1475', 'Expenditure_BWX', 'Base_8511', 'Infraction_JYZB', 'Infraction_ZTYG', 'Infraction_EYU', 'Expenditure_UWVG', 'Infraction_CZE', 'Base_65352', 'Risk_7095', 'Infraction_JBR', 'Base_36516', 'Risk_6346', 'Expenditure_HRQ', 'Risk_2102', 'Base_7331', 'Infraction_XWX', 'Risk_4553', 'Risk_8742', 'Infraction_VHU', 'Risk_4247', 'Risk_2380', 'Infraction_GSS', 'Expenditure_HKXV', 'Infraction_MHM', 'Risk_4160', 'Risk_3506', 'Base_23737', 'Expenditure_GCAO', 'Risk_9367', 'Base_7910', 'Expenditure_GMC', 'Risk_9423',

In [12]:
import numpy as np
import pandas as pd

# Función para aplicar transformaciones (raíz cuadrada o logaritmo)
def apply_transformations(df, sqrt_cols, log_cols):
    """
    Aplica transformaciones a columnas seleccionadas:
    - Raíz cuadrada para sqrt_cols
    - Logaritmo natural (log1p) para log_cols
    """
    for col in sqrt_cols:
        if col in df.columns:
            df[col] = np.sqrt(df[col].clip(lower=0))
            print(f"✔ Raíz cuadrada aplicada en '{col}'.")
        else:
            print(f"❌ Columna '{col}' no encontrada para raíz cuadrada.")

    for col in log_cols:
        if col in df.columns:
            df[col] = np.log1p(df[col].clip(lower=0))
            print(f"✔ Logaritmo aplicado en '{col}'.")
        else:
            print(f"❌ Columna '{col}' no encontrada para logaritmo.")
    return df

# Función para convertir columnas categóricas con enteros
def convert_to_category(df, columns):
    """
    Convierte columnas seleccionadas a tipo 'category'.
    """
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype('category')
            print(f"✔ Columna '{col}' convertida a categoría.")
        else:
            print(f"❌ Columna '{col}' no encontrada para conversión a categoría.")
    return df

# Función para convertir columnas booleanas a enteros
def convert_booleans_to_int(df):
    """
    Convierte columnas booleanas en el DataFrame a tipo int.
    """
    boolean_columns = df.select_dtypes(include=['bool']).columns
    if len(boolean_columns) > 0:
        df[boolean_columns] = df[boolean_columns].astype(int)
        print(f"✔ Columnas booleanas convertidas a int: {list(boolean_columns)}")
    else:
        print("❌ No se encontraron columnas booleanas.")
    return df

# Función para eliminar columnas constantes
def drop_constant_columns(df, columns_to_drop=None):
    """
    Elimina columnas con valores constantes o columnas especificadas.
    """
    if columns_to_drop:
        for col in columns_to_drop:
            if col in df.columns:
                df = df.drop(columns=[col])
                print(f"✔ Columna '{col}' eliminada.")
            else:
                print(f"❌ Columna '{col}' no encontrada.")

    constant_columns = [col for col in df.columns if df[col].nunique() <= 1]
    if constant_columns:
        df = df.drop(columns=constant_columns)
        print(f"✔ Columnas constantes eliminadas: {constant_columns}")
    else:
        print("❌ No se encontraron columnas constantes.")
    return df

# Columnas recomendadas para transformaciones
sqrt_columns = [
    'Risk_1930', 'Infraction_SNZ', 'Infraction_QJJF', 'Infraction_FMXQ',
    'Expenditure_FIP', 'Base_0580', 'Base_91828', 'Expenditure_IDZ',
    'Base_22178', 'Infraction_ZVW', 'Infraction_JBR', 'Expenditure_UIWS',
    'Expenditure_MTRQ', 'Expenditure_HPM', 'Infraction_HUK', 'Infraction_QWWW',
    'Infraction_QGR', 'Infraction_LSX', 'Infraction_DNOU'
]
log_columns = ['Infraction_BGGU']

# Columnas categóricas y booleanas
categorical_int_columns = ['Infraction_NMCB', 'Infraction_ZRH', 'Infraction_WIS']
columns_to_drop = ['Expenditure_AHF', 'Base_23737']

# Flujo Principal
# 1. Aplicar transformaciones
df = apply_transformations(df, sqrt_columns, log_columns)

# 2. Convertir columnas categóricas
df = convert_to_category(df, categorical_int_columns)

# 3. Convertir booleanos a enteros
df = convert_booleans_to_int(df)

# 4. Eliminar columnas constantes o específicas
df = drop_constant_columns(df, columns_to_drop)

# 5. Verificación final
print("\nTipos de datos actualizados:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum().sort_values(ascending=False))

# Mostrar primeras filas
print("\nPrimeras filas del DataFrame actualizado:")
print(df.head())

✔ Raíz cuadrada aplicada en 'Risk_1930'.
✔ Raíz cuadrada aplicada en 'Infraction_SNZ'.
✔ Raíz cuadrada aplicada en 'Infraction_QJJF'.
✔ Raíz cuadrada aplicada en 'Infraction_FMXQ'.
✔ Raíz cuadrada aplicada en 'Expenditure_FIP'.
✔ Raíz cuadrada aplicada en 'Base_0580'.
✔ Raíz cuadrada aplicada en 'Base_91828'.
✔ Raíz cuadrada aplicada en 'Expenditure_IDZ'.
✔ Raíz cuadrada aplicada en 'Base_22178'.
✔ Raíz cuadrada aplicada en 'Infraction_ZVW'.
✔ Raíz cuadrada aplicada en 'Infraction_JBR'.
✔ Raíz cuadrada aplicada en 'Expenditure_UIWS'.
✔ Raíz cuadrada aplicada en 'Expenditure_MTRQ'.
✔ Raíz cuadrada aplicada en 'Expenditure_HPM'.
✔ Raíz cuadrada aplicada en 'Infraction_HUK'.
✔ Raíz cuadrada aplicada en 'Infraction_QWWW'.
✔ Raíz cuadrada aplicada en 'Infraction_QGR'.
✔ Raíz cuadrada aplicada en 'Infraction_LSX'.
✔ Raíz cuadrada aplicada en 'Infraction_DNOU'.
✔ Logaritmo aplicado en 'Infraction_BGGU'.
✔ Columna 'Infraction_NMCB' convertida a categoría.
✔ Columna 'Infraction_ZRH' convertida 

In [13]:
import pandas as pd

# Lista de columnas seleccionadas
rfe_columns = [
    'Payment_6804', 'Base_80863', 'Expenditure_JIG', 'Base_02683',
    'Infraction_ZWWJ', 'Infraction_QJJF', 'Infraction_EJZ',
    'Infraction_GGO', 'Infraction_TLPJ', 'Base_1165', 'Base_39598',
    'Base_6187', 'Base_85131', 'Risk_9995', 'Infraction_AYWV', 'Base_9516',
    'Expenditure_HMO', 'Infraction_BSU', 'Infraction_ZYW', 'Infraction_TBP',
    'Infraction_PBC', 'Base_0229', 'Base_69608', 'Base_3041',
    'Infraction_QKZN', 'Infraction_CZE', 'Expenditure_MTRQ',
    'Infraction_XEPQ', 'Infraction_RKTA', 'Infraction_KEJT'
]

# 1. Verificar que todas las columnas existen en el DataFrame
missing_columns = [col for col in rfe_columns if col not in df.columns]

if missing_columns:
    print("❌ Las siguientes columnas no existen en el DataFrame:")
    print(missing_columns)
    # Eliminar columnas que no existen para evitar errores
    rfe_columns = [col for col in rfe_columns if col in df.columns]
else:
    print("✔ Todas las columnas existen en el DataFrame.")

# 2. Crear un nuevo DataFrame con las columnas validadas
if rfe_columns:
    df_rfe = df[rfe_columns].copy()
    print(f"✔ DataFrame creado con {len(rfe_columns)} columnas seleccionadas.")
    print(df_rfe.head())
else:
    print("❌ No se encontraron columnas válidas para crear el DataFrame.")

# 3. Exportar el DataFrame a CSV
output_file = 'processed_dataset.csv'
try:
    df_rfe.to_csv(output_file, index=False)
    print(f"✔ DataFrame exportado exitosamente a '{output_file}'.")
except Exception as e:
    print(f"❌ Error al exportar el archivo CSV: {e}")

✔ Todas las columnas existen en el DataFrame.
✔ DataFrame creado con 30 columnas seleccionadas.
                                                    Payment_6804  Base_80863  \
ID                                                                             
33333337027237328070743333423202763537707325232...      0.938469    1.006838   
33333337027237328070743333423202763537707325232...      0.936665    1.000653   
33333337027237328070743333423202763537707325232...      0.954180    1.009672   
33333337027237328070743333423202763537707325232...      0.960384    1.002700   
33333337027237328070743333423202763537707325232...      0.947248    1.000727   

                                                    Expenditure_JIG  \
ID                                                                    
33333337027237328070743333423202763537707325232...         0.124035   
33333337027237328070743333423202763537707325232...         0.126750   
33333337027237328070743333423202763537707325232...         

In [19]:
import joblib
import os
import numpy as np
from sklearn.impute import SimpleImputer

# Verificar si los archivos existen
scaler_file = "scaler.joblib"
model_file = "knn_model.joblib"

if os.path.exists(scaler_file) and os.path.exists(model_file):
    scaler = joblib.load(scaler_file)
    knn_model = joblib.load(model_file)
    print("✔ Scaler y modelo k-NN cargados exitosamente.")
else:
    raise FileNotFoundError("❌ Archivos 'scaler.joblib' o 'knn_model.joblib' no encontrados.")

# 1. Preparar las características
if 'label' in df_rfe.columns:
    df_rfe = df_rfe.drop(columns=['label'])

# Excluir 'ID' si existe
X_rfe = df_rfe.drop(columns=['ID'], errors='ignore')

# 2. Imputar valores faltantes (NaN) usando la mediana
print("🔄 Imputando valores faltantes...")
imputer = SimpleImputer(strategy='median')  # Puedes cambiar 'median' por 'mean' si lo prefieres
X_rfe_imputed = imputer.fit_transform(X_rfe)

# Verificar si la imputación fue exitosa
if np.isnan(X_rfe_imputed).sum() == 0:
    print("✔ Todos los valores NaN han sido imputados correctamente.")
else:
    raise ValueError("❌ Persisten valores NaN después de la imputación.")

# 3. Escalar los datos imputados
try:
    X_rfe_scaled = scaler.transform(X_rfe_imputed)
    print("✔ Datos escalados correctamente.")
except Exception as e:
    raise RuntimeError(f"❌ Error al escalar los datos: {e}")

# 4. Realizar predicciones con el modelo k-NN
try:
    predictions = knn_model.predict(X_rfe_scaled)
    print("✔ Predicciones realizadas con éxito.")
except Exception as e:
    raise RuntimeError(f"❌ Error al predecir con el modelo k-NN: {e}")

# 5. Agregar las predicciones como columna 'label'
df_rfe = df_rfe.reset_index(drop=True)
df_rfe['label'] = predictions
print("✔ Columna 'label' añadida al DataFrame.")

# 6. Crear columna 'ID' si no existe
if 'ID' not in df_rfe.columns:
    print("🔄 Creando la columna 'ID' a partir del índice.")
    df_rfe['ID'] = range(1, len(df_rfe) + 1)

# 7. Crear DataFrame final y exportar
df_rfe_final = df_rfe[['ID', 'label']]
print("\nPrimeras filas del DataFrame final:")
print(df_rfe_final.head())

output_file = 'test_labels.csv'
df_rfe_final.to_csv(output_file, index=False)
print(f"✔ Archivo exportado exitosamente como '{output_file}'.")

✔ Scaler y modelo k-NN cargados exitosamente.
🔄 Imputando valores faltantes...
✔ Todos los valores NaN han sido imputados correctamente.
✔ Datos escalados correctamente.
✔ Predicciones realizadas con éxito.
✔ Columna 'label' añadida al DataFrame.

Primeras filas del DataFrame final:
   ID  label
0   1      0
1   2      0
2   3      0
3   4      0
4   5      0
✔ Archivo exportado exitosamente como 'test_labels.csv'.
